<h1> imports

In [1]:
from my_package.text_processing import *
from my_package.summary_evalution import *

In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import tqdm as tqdm
import itertools
#lda
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

In [3]:
import math
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from lexrank.utils.text import tokenize
from lexrank.mappings.stopwords import STOPWORDS

In [4]:
# f = open("SummaryInput.txt", "r", encoding="utf8")
# text = ""
# for line in f:
#     text += line.replace('!','.').replace('?','.').replace('\n',' ')
# f.close()
# filtered_sentences,sentences = tokenization(text)  # Tokenize the text into sentences

In [5]:
# out = text_matching(filtered_sentences, sentences)
# buildDF(filtered_sentences, sentences)

In [6]:
# ensemble_dic = getTextRanks(filtered_sentences, sentences)

In [7]:
# ensemble_dic[0][2]

<h1> Pre Processing </h1>

In [8]:
def tokenization(text):
    sentences = sent_tokenize(text)
    filtered_sentences = remove_stop_words(sentences)
    return filtered_sentences,sentences

In [9]:
def remove_stop_words(sentences):
    # define a set of stop words 
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []
    for sentence in sentences:
        # tokenize each sentence into words
        word_tokens = word_tokenize(sentence)
        # remove all stop words from each sentence
        filtered_words = [word for word in word_tokens if not word in stop_words]
        # join all filtered words back into a single sentence
        filtered_sentence = ' '.join(filtered_words)
        filtered_sentences.append(filtered_sentence)
    return filtered_sentences

In [10]:
def normalization(lst):
    maxScore = max(lst)    
    sortedScored = list(reversed(sorted(lst)))
    
    for i in range(len(lst)):
        sentence_map[i].append(lst[i] / maxScore)
        sentence_map[i].append(sortedScored.index(sent_scores[i]))

<h1> Text Matching 

In [11]:
def text_matching(filtered_sentences, sentences):
    word_frequencies = {}  # Create an empty dictionary to store the word frequencies
    for sentence in filtered_sentences:  # Loop through each sentence in the text
        words = nltk.word_tokenize(sentence)  # Tokenize each sentence into words
        for word in words:  # Loop through each word in the sentence
            if word not in word_frequencies.keys():  # Check if the word is already in the dictionary
                word_frequencies[word] = 1  # If not, set its count to 1
            else:
                word_frequencies[word] += 1  # If yes, increment its count by 1
    summary = []  # Create an empty list to store the summary sentences
    sentence_map = {} # Create an empty dictionary to store the sentence scores
    sent_scores = []
    sent_index = 0
    for sentence in sentences:  # Loop through each sentence in the text again
        words = nltk.word_tokenize(sentence)  # Tokenize each sentence into words again
        score = 0  # Initialize a score variable to 0
        for word in words:  # Loop through each word in the sentence
            if word in word_frequencies.keys():  # Check if the current word is present in our dictionary of word frequencies
                score += word_frequencies[word]  # If yes, add its frequency to our score variable
        sentence_map[sent_index] = [sentence,score]
        sent_scores.append(score)
        sent_index += 1
    return sentence_map

<h1> Luhn

In [12]:
def luhn_algorithm(filtered_sentences, sentences):
    # timeNow("Starting Luhn")
    # Initialize a list to store the summary
    summary = []
    # # Split the text into sentences
    # sentences = text.split('.')
    # Initialize a list to store the sentence scores
    sentence_scores = []
    sentence_map = {}
    sent_index = 0
    # Iterate through each sentence
    for sentence in filtered_sentences:
        # Split the sentence into words
        words = sentence.split()
        # Initialize a score for the sentence
        score = 0
        # Iterate through each word
        for word in words:
            # Calculate the score for the word
            score += len(word)
        # Add the score to the sentence scores list
        sentence_scores.append(score)
        sentence_map[sent_index] = [sentences[sent_index],score]
        sent_index += 1
    return sentence_map

<h1> Latent Semantic Analysis

In [13]:
def create_tf_idf(filtered_sentences):
    tfidfconverter = TfidfVectorizer()
    X = tfidfconverter.fit_transform(filtered_sentences).toarray()
    return X

In [14]:
def lsa_algorithm(X):
    svdmodel = TruncatedSVD(n_components=2)
    svdmodel.fit(X)
    result = svdmodel.transform(X)
    return result

In [15]:
def lsa_summarization(filtered_sentences, sentences):
    # timeNow("Starting LSA")
    # sentences = tokenization(text)
    # filtered_sentences = remove_stop_words(sentences)
    X = create_tf_idf(filtered_sentences)
    result = lsa_algorithm(X)
    scores = result[:,1]
    summary = ""
    sentence_map = {}
    for i in range (len (scores)):
        sentence_map[i] = [sentences[i], scores[i]]
    # timeNow("Finished LSA")
    return sentence_map

<h1> Text Rank

In [16]:
word_embeddings = {}

In [17]:
if not(word_embeddings):
    print("Loading Big File...")
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    print("Finished Loading Big File.")
def textRank(filtered_sentences, sentences):
    # timeNow("Starting Text Rank")
    sentence_vectors = []
    for i in filtered_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100))
        sentence_vectors.append(v)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]   
    #begining Graph stap
    import networkx as nx
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    sent_map = {}
    for index in range(len(sentences)):
        sent_map[index] = [sentences[index] , scores[index]]
    #ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # timeNow("Finished Text Rank")
    return sent_map

Loading Big File...
Finished Loading Big File.


<h1> Lex Rank </h1>

In [18]:
def LexRank_algorithm(filtered_sentences,sentences,size=5,threshold = 0.095):
    tfidfconverter = TfidfVectorizer()
    tf_idf = tfidfconverter.fit_transform(filtered_sentences).toarray()
    length = len(tf_idf)
    similarity_matrix = np.zeros([length] * 2)
    
    for i in range(length):
        for j in range(i, length):
            similarity = cosine_similarity(tf_idf[i],tf_idf[j],i,j)

            if similarity:
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity    
    
    def get_summary(sentences,similarity_matrix,threshold,summary_size=1):

        if not isinstance(summary_size, int) or summary_size < 1:
            raise ValueError('\'summary_size\' should be a positive integer')

        lex_scores = rank_sentences(sentences,similarity_matrix,threshold)

        sorted_ix = np.argsort(lex_scores)[::-1]

        summary_index=[]
        for i in sorted_ix[:summary_size]:
            summary_index.append(i)
        #print(summary_index)
        return lex_scores,summary_index

    scores , summary_index = get_summary(sentences,similarity_matrix,threshold,size)

    sentence_map = {}
    for i in range (len (scores)):
        sentence_map[i] = [sentences[i], scores[i]]
    return sentence_map

def connected_nodes(matrix):
    _, labels = connected_components(matrix)
    z = csr_matrix(matrix)
    groups = []
    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)
    return groups

def cosine_similarity(list_1, list_2,i,j):
        if i == j :
            return 1
        dot = np.dot(list_1, list_2)
        if math.isclose(dot, 0):
            return 0
        norm = (np.linalg.norm(list_1) * np.linalg.norm(list_2))
        cos_sim = dot / norm
        return cos_sim
    
    
def stationary_distribution(transition_matrix,normalized=True):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)
    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix)
        distribution[group] = eigenvector
    if normalized:
        distribution /= n_1
    return distribution

def _power_method(transition_matrix):
    sentences_count = len(transition_matrix)
    eigenvector = np.ones(sentences_count)
    if len(eigenvector) == 1:
        return eigenvector
    transposed_matrix = transition_matrix.T
    lambda_val = 1.0

    while np.allclose(lambda_val, eigenvector):
        eigenvector_next = np.dot(transposed_matrix, eigenvector)
        lambda_val = np.linalg.norm(np.subtract(eigenvector_next, eigenvector))
        eigenvector = eigenvector_next
    return eigenvector


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    return weights_matrix / row_sum

def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = weights_matrix#np.zeros(weights_matrix.shape)
    #print(discrete_weights_matrix)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1
    #print(discrete_weights_matrix)

    return create_markov_matrix(discrete_weights_matrix)

def degree_centrality_scores(similarity_matrix,threshold=None,increase_power=True):
    if not (threshold is None or isinstance(threshold, float) and 0 <= threshold < 1):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None')

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(similarity_matrix,threshold)

    scores = stationary_distribution(markov_matrix,normalized=True)
    return scores

def rank_sentences(sentences,similarity_matrix,threshold=0.03):  
    scores = degree_centrality_scores(similarity_matrix,threshold)
    return scores

<h1> LDA </h1>

In [19]:
def lda_algorithm(filtered_sentences,sentences,num_topics = 3):
    # Create a CountVectorizer object to preprocess the text
    vectorizer = CountVectorizer(stop_words='english')

    # Fit and transform the vectorizer to the text
    doc_term_matrix = vectorizer.fit_transform(filtered_sentences)

    # Create an LDA object using Gibbs sampling
    lda = LatentDirichletAllocation(n_components=num_topics, learning_method='batch', random_state=42)

    # Fit the LDA model to the document-term matrix
    lda.fit(doc_term_matrix)

    # Extract the topic summaries
    topic_word_distributions = lda.components_
    for i, topic_dist in enumerate(topic_word_distributions):
        topic_words = [vectorizer.get_feature_names_out()[j] for j in topic_dist.argsort()[:-6:-1]]
        #print(f"Topic {i}: {' '.join(topic_words)}")

    # Extract the topic distribution for each sentence
    sentence_topic_distributions = lda.transform(doc_term_matrix)

    # Calculate the sentence scores
    sentence_scores = sentence_topic_distributions.max(axis=1)

    # Sort the sentences by score in descending order
    sorted_indices = sentence_scores.argsort()[::-1]

    # Extract the top N sentences
    # top_indices = sorted_indices[:size]
    # top_sentences = [sentences[i] for i in top_indices]

    # Print the summary
    #print('\n'.join(top_sentences))
    sent_map = {}
    for i in range(len(sentences)):
        sent_map[i] = [sentences[i],sentence_scores[i],sorted_indices[i]]
        
    return sent_map

<h1> Summarizing the dataset

In [20]:
def createEnsembledic(tm_dic,luhn_dic,lsa_dic,tr_dic,lex_dic, lda_dic):
    ensemble_dic = {}
    finalScores = []
    for key in tm_dic:
        final_score = tm_dic[key][1] + luhn_dic[key][1]+ lsa_dic[key][1] + tr_dic[key][1] + lex_dic[key][1] + lda_dic[key][1]
        finalScores.append(final_score)
        scores_arr = [tm_dic[key][1],luhn_dic[key][1],lsa_dic[key][1],tr_dic[key][1],lex_dic[key][1],lda_dic[key][1]]
        sent = tm_dic[key][0]
        ensemble_dic[key] = [final_score, scores_arr, sent]
    return ensemble_dic,finalScores

In [21]:
def ensembleSummary(ensemble_dic,finalScores,size):
    finalScores = list(reversed(sorted(finalScores)))[:]
    summaryScores = finalScores[:size] # number of summary sentences
    summary = []
    for key in ensemble_dic:
        if(ensemble_dic[key][0] in summaryScores):
            summary.append(ensemble_dic[key][2])
    return summary

In [22]:
def extractSummaryFromDic(dic, size):
    summaryScores = []
    summarySent = []
    for i in range(size):
        summaryScores.append(dic[i][1])
        summarySent.append(dic[i][0])
        
    for i in range(size, len(dic), 1):
        if(dic[i][1] > min(summaryScores)):
            ind = summaryScores.index(min(summaryScores))
            summaryScores[ind] = dic[i][1]
            summarySent[ind] = dic[i][0]
    return summarySent

In [23]:
def getTextRanks(filtered_sentences, sentences):
    tm_dic = text_matching(filtered_sentences, sentences)
    luhn_dic = luhn_algorithm(filtered_sentences, sentences)
    lsa_dic = lsa_summarization(filtered_sentences, sentences)
    tr_dic = textRank(filtered_sentences, sentences)
    lex_dic = LexRank_algorithm(filtered_sentences, sentences)
    lda_dic = lda_algorithm(filtered_sentences, sentences)
    ensemble_dic, ensemble_scores = createEnsembledic(tm_dic, luhn_dic, lsa_dic, tr_dic, lex_dic, lda_dic)
    return ensemble_dic#, ensemble_scores, tm_dic, luhn_dic, lsa_dic, tr_dic

In [24]:
def get_combinations(arr):
    combinations = []
    for i in range(1,len(arr)+1):
        for element in itertools.combinations(arr, i):
            combinations.append(element)
    return combinations

In [25]:
def allCombs(ensemble_dic, useWeights, weights):
    Algorithms = ["Sent","Tm","Lex","Luhn","Lsa","Tr","LDA"]
    lst = []
    lstCols = Algorithms #get_combinations(Algorithms)
    wtm = 1
    wlex = 1
    wluhn = 1
    wlsa = 1
    wtr = 1
    wlda = 1
    if(useWeights):
        wtm = weights["tm"]
        wlex =  weights["lex"]
        wluhn =  weights["luhn"]
        wlsa =  weights["lsa"]
        wtr =  weights["tr"]
        wlda =  weights["lda"]
    for i in ensemble_dic:
        tm = ensemble_dic[i][1][0] * wtm
        luhn = ensemble_dic[i][1][1] * wluhn
        lsa = ensemble_dic[i][1][2] * wlsa
        tr = ensemble_dic[i][1][3] * wtr
        lex = ensemble_dic[i][1][4] * wlex
        lda = ensemble_dic[i][1][5] * wlda
        values = [ensemble_dic[i][2],tm,lex,luhn,lsa,tr,lda]
        lst.append(values)
        # CombNums = get_combinations(values)
        # for i,comb in enumerate(CombNums):
        #     CombNums[i] = sum(comb)/len(comb)
        # lst.append(CombNums)
    combinationScores = pd.DataFrame(lst, columns = lstCols)     
    return combinationScores

In [26]:
def allCombsDf(df, useWeights = False, weights = []):
    Algorithms = ["Tm","Lex","Luhn","Lsa","Tr","LDA"]
    lst = []
    lstCols = get_combinations(Algorithms)
    wtm = 1
    wlex = 1
    wluhn = 1
    wlsa = 1
    wtr = 1
    wlda = 1
    if(useWeights):
        wtm = weights["tm"]
        wlex =  weights["lex"]
        wluhn =  weights["luhn"]
        wlsa =  weights["lsa"]
        wtr =  weights["tr"]
        wlda =  weights["lda"]
    for row in df.iterrows():
        tm = row[1][Algorithms[0]] * wtm
        luhn = row[1][Algorithms[1]] * wluhn
        lsa = row[1][Algorithms[2]] * wlsa
        tr = row[1][Algorithms[3]] * wtr
        lex = row[1][Algorithms[4]] * wlex
        lda = row[1][Algorithms[5]] * wlda
        values = [tm,lex,luhn,lsa,tr,lda]

        CombNums = get_combinations(values)
        for i,comb in enumerate(CombNums):
            CombNums[i] = sum(comb)/len(comb)
        lst.append(CombNums)
    combinationScores = pd.DataFrame(lst, columns = lstCols)     
    return combinationScores

In [27]:
def buildDF(filtered_sentences, sentences, useWeights = False, weights = []):
    ensemble_dic = getTextRanks(filtered_sentences, sentences)
    df = allCombs(ensemble_dic, useWeights, weights)
    return df

In [28]:
def summarizeWith(sentences, df, algorithm, percentage):
    if(percentage >= 1):
        sent = int(percentage)
    else:
        sent = int(percentage * len(df))
    summInds = df.nlargest(n=sent, columns=[algorithm])[algorithm].keys()
    # summInds = sorted(summInds)
    summ = ""
    for i in summInds:
        summ += sentences[i]
    return summ

In [29]:
# with open ("summary input.txt", "r") as myfile:
#     data = myfile.read()
#     filtered_sentences,sentences = tokenization(data)

In [30]:
# buildDF(filtered_sentences, sentences)